Модель "Random forest"

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Загрузка тренировочных данных
train_data = pd.read_csv('cleaned_train.csv')

# Разделение признаков и целевой переменной
X_train = train_data.drop(columns=['client_id', 'target'])
y_train = train_data['target']

# Преобразование категориальных признаков
X_train = pd.get_dummies(X_train, drop_first=True)

# Обучение модели Random Forest
model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    n_estimators=100,  # Количество деревьев
    max_depth=10,      # Максимальная глубина деревьев
    min_samples_split=5,  # Минимальное количество выборок для разбиения
)
model.fit(X_train, y_train)

# Оценка качества модели на тренировочной выборке
y_pred_train = model.predict(X_train)
print("Отчет на обучающей выборке:")
print(classification_report(y_train, y_pred_train))

roc_auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print(f"ROC-AUC на обучающей выборке: {roc_auc_train:.3f}")

# Предсказание вероятностей
y_proba_train = model.predict_proba(X_train)[:, 1]

# Фильтрация потенциальных покупателей (вероятность >= 0.6)
threshold = 0.6
potential_buyers_train = train_data[y_proba_train >= threshold]

# Сохранение потенциальных покупателей в файл
potential_buyers_train.to_csv('potential_buyers_train_rf.csv', index=False)
print(f"Сохранено {len(potential_buyers_train)} потенциальных покупателей в файл 'potential_buyers_train_rf.csv'.")

C:\Users\b1gtz\AppData\Local\Temp\ipykernel_6436\748764393.py:6: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('cleaned_train.csv')


Отчет на обучающей выборке:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96     14020
           1       0.28      0.94      0.44       436

    accuracy                           0.93     14456
   macro avg       0.64      0.93      0.70     14456
weighted avg       0.98      0.93      0.94     14456

ROC-AUC на обучающей выборке: 0.975
Сохранено 109 потенциальных покупателей в файл 'potential_buyers_train_rf.csv'.


Модель "Cat boost"

In [4]:
!pip install catboost


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\b1gtz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Загрузка тренировочных данных
train_data = pd.read_csv('cleaned_train.csv')

# Разделение признаков и целевой переменной
X_train = train_data.drop(columns=['client_id', 'target'])
y_train = train_data['target']

# Определение категориальных признаков
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Преобразование категориальных признаков в строки
X_train[categorical_features] = X_train[categorical_features].astype(str)

# Обучение модели CatBoost
model = CatBoostClassifier(
    iterations=500,            # Количество итераций
    depth=8,                   # Глубина дерева
    learning_rate=0.05,        # Скорость обучения
    loss_function='Logloss',   # Функция потерь
    class_weights=[1, 20],     # Сбалансировка классов
    random_seed=42,
    verbose=100                # Периодичность логирования
)
model.fit(X_train, y_train, cat_features=categorical_features)

# Оценка качества модели на тренировочной выборке
y_pred_train = model.predict(X_train)
print("Отчет на обучающей выборке:")
print(classification_report(y_train, y_pred_train))

roc_auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print(f"ROC-AUC на обучающей выборке: {roc_auc_train:.3f}")

# Предсказание вероятностей
y_proba_train = model.predict_proba(X_train)[:, 1]

# Фильтрация потенциальных покупателей (вероятность >= 0.6)
threshold = 0.6
potential_buyers_train = train_data[y_proba_train >= threshold]

# Сохранение потенциальных покупателей в файл
potential_buyers_train.to_csv('potential_buyers_train_cb.csv', index=False)
print(f"Сохранено {len(potential_buyers_train)} потенциальных покупателей в файл 'potential_buyers_train_cb.csv'.")

C:\Users\b1gtz\AppData\Local\Temp\ipykernel_8348\392297685.py:6: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('cleaned_train.csv')


0:	learn: 0.5647702	total: 103ms	remaining: 51.5s
100:	learn: 0.0396099	total: 11.4s	remaining: 45.1s
200:	learn: 0.0257123	total: 21.8s	remaining: 32.4s
300:	learn: 0.0159262	total: 34.5s	remaining: 22.8s
400:	learn: 0.0112098	total: 47.4s	remaining: 11.7s
499:	learn: 0.0092482	total: 60s	remaining: 0us
Отчет на обучающей выборке:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14020
           1       0.99      1.00      0.99       436

    accuracy                           1.00     14456
   macro avg       0.99      1.00      1.00     14456
weighted avg       1.00      1.00      1.00     14456

ROC-AUC на обучающей выборке: 1.000
Сохранено 439 потенциальных покупателей в файл 'potential_buyers_train_cb.csv'.
